In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import  re
pd.set_option('display.max_columns', 50)

In [ ]:
#Reading in Model List
model_list = pd.read_csv("../input/model-list/model_list_20200825 (1).csv")
#Cell Line Mapping
cell_line_mapping = model_list[['model_id','sample_id','model_name','COSMIC_ID','BROAD_ID','CCLE_ID','tissue','tissue_status','cancer_type']].copy()
#cell_line_mapping.shape
cell_line_mapping.head()

In [ ]:
#Reading in Gene Mapping
gene_mapping= pd.read_csv(r'../input/gene-mapping-updated/Gene Mapping_20191101.csv')
print(gene_mapping.shape)
gene_mapping.head()

In [ ]:
#readings metabolomics data
ccle_metabolomics_original = pd.read_csv(r'../input/ccle-metabolomics/CCLE_metabolomics_20190502 (3).csv')
ccle_metabolomics = ccle_metabolomics_original.copy()
print(ccle_metabolomics.shape)
ccle_metabolomics.head()

In [ ]:
# map CCLE_ID to DepMap_ID
ccle_metabolomics = ccle_metabolomics.merge(cell_line_mapping[['CCLE_ID','model_id']],
                     left_on = 'CCLE_ID',
                     right_on = 'CCLE_ID',
                     how = 'left')
print('percentage of na in model_id: ', ccle_metabolomics['model_id'].isna().sum()/ccle_metabolomics.shape[0])
nan_count = ccle_metabolomics['model_id'].isna().sum()
print(nan_count)

print(ccle_metabolomics.shape)

#Drop missing values in column 'model_id'
ccle_metabolomics.dropna(subset=['model_id'], inplace=True) 

print(ccle_metabolomics.shape)
ccle_metabolomics.head()

In [ ]:
#Remove CCLE_ID & DepMap_ID
matrix_metabolomics = ccle_metabolomics.iloc[:,2:]
matrix_metabolomics = matrix_metabolomics[[list(matrix_metabolomics.columns)[-1]]+ list(matrix_metabolomics.columns[:-1])]

matrix_metabolomics.to_csv('./matrix_metabolomics.csv', index = False)
matrix_metabolomics.to_csv('./matrix_metabolomics_pca.csv', header=False, index = False)

print(matrix_metabolomics.shape)
matrix_metabolomics.head()

In [ ]:
ccle_rppa_original = pd.read_csv(r'../input/ccle-rppa/CCLE_RPPA_20181003 (3).csv')
ccle_rppa = ccle_rppa_original.copy()
print(ccle_rppa.shape)

# map CCLE_ID to model_id
ccle_rppa.rename(columns={'Unnamed: 0':'CCLE_ID'},inplace=True)
matrix_rppa = ccle_rppa.merge(cell_line_mapping[['CCLE_ID','model_id']],
                     left_on = 'CCLE_ID',
                     right_on = 'CCLE_ID',
                     how = 'left')
#Removing NA values (meaning no model ID matching)

print('percentage of na in model_id: ', matrix_rppa['model_id'].isna().sum()/matrix_rppa.shape[0])
nan_count = matrix_rppa['model_id'].isna().sum()
print(nan_count)

print(matrix_rppa.shape)
matrix_rppa.dropna(subset=['model_id'], inplace=True)
print(matrix_rppa.shape)

matrix_rppa = matrix_rppa.iloc[:,1:]
matrix_rppa = matrix_rppa[[list(matrix_rppa.columns)[-1]]+ list(matrix_rppa.columns[:-1])]
matrix_rppa.to_csv('./matrix_rppa.csv', index = False)
matrix_rppa.to_csv('./matrix_rppa_pca.csv', header=False, index = False)

# In[23]:


print(matrix_rppa.shape)
matrix_rppa.head()


# In[24]:


#ccle_rppa_original.head()




In [ ]:
gdsc_cnv_original = pd.read_csv(r'../input/gene-cnv-2018-1113-1545/gene_cnv_2018-11-13_1545.csv')
gdsc_cnv = gdsc_cnv_original.copy()
print(gdsc_cnv.shape)

matrix_cnv = gdsc_cnv[['model_id','gene_id','abs_copy_number']]
print(matrix_cnv.shape)

matrix_cnv.drop_duplicates(inplace = True)
print(matrix_cnv.shape)

matrix_cnv.dropna(inplace = True)
print(matrix_cnv.shape)
matrix_cnv = matrix_cnv.pivot_table(index = 'model_id',
                          columns = 'gene_id',
                        values = 'abs_copy_number')
print(matrix_cnv.shape)
matrix_cnv.to_csv('./matrix_cnv.csv', index = True)
matrix_cnv.to_csv('./matrix_cnv_pca.csv', header=False, index = False)


#To count no. of NAN in df
# print('NAN values',matrix_cnv.isnull().sum().sum())
#gdsc_cnv_original.head()

matrix_cnv.head()

In [ ]:
ccle_expression_original = pd.read_csv(r'../input/ccle-expression-0920/CCLE_expression_ 0920.csv')
ccle_expression = ccle_expression_original.copy()
print(ccle_expression.shape)
ccle_expression.head()

In [ ]:
# map BROAD_ID to model_id
ccle_expression.rename(columns={'Unnamed: 0':'BROAD_ID'},inplace=True)
ccle_expression = ccle_expression.merge(cell_line_mapping[['BROAD_ID','model_id']],
                     left_on = 'BROAD_ID',
                     right_on = 'BROAD_ID',
                     how = 'left')

print(ccle_expression.shape)

nan_count = ccle_expression['model_id'].isna().sum()
print(nan_count)

ccle_expression.dropna(subset=['model_id'], inplace=True)
print(ccle_expression.shape)

matrix_expression = ccle_expression.iloc[:,1:]
matrix_expression = matrix_expression[[list(matrix_expression.columns)[-1]]+ list(matrix_expression.columns[:-1]) ]

In [ ]:
# map cosmic_gene_symbol to gene_id
matrix_expression_t = matrix_expression.T
matrix_expression_t.reset_index(level=0, inplace=True)
matrix_expression_t['index'] = matrix_expression_t['index'].str.split(' ', expand = True)[0]
matrix_expression_t = matrix_expression_t.merge(gene_mapping[['cosmic_gene_symbol','gene_id']],
                     left_on = 'index',
                     right_on = 'cosmic_gene_symbol',
                     how = 'left')

matrix_expression_t['gene_id'][0] = 'model_id'

matrix_expression.columns = matrix_expression_t['gene_id']
matrix_expression = matrix_expression.iloc[:,matrix_expression.columns.notna()]
matrix_expression.to_csv('./matrix_expression.csv', index = False)
matrix_expression.to_csv('./matrix_expression_pca.csv', header=False, index = False)

In [ ]:
print(matrix_expression.shape)
matrix_expression.head()

#ccle_expression.head()

In [ ]:
ccle_mutations_original = pd.read_csv(r'../input/ccle-mutations-comma-delimited/CCLE_mutations_comma_delimited.csv')
ccle_mutations = ccle_mutations_original.copy()
print(ccle_mutations.shape)
ccle_mutations.head()

In [ ]:
# simplify variant classification
ccle_mutations['Variant_Classification_Simplified'] = ccle_mutations['Variant_Classification']
ccle_mutations['Variant_Classification_Simplified'] = ccle_mutations['Variant_Classification_Simplified'].str.replace("Missense_Mutation","Missense")
ccle_mutations['Variant_Classification_Simplified'] = ccle_mutations['Variant_Classification_Simplified'].str.replace("Silent","Silent")
ccle_mutations['Variant_Classification_Simplified'] = ccle_mutations['Variant_Classification_Simplified'].str.replace("Splice_Site","Splice_Site")
ccle_mutations['Variant_Classification_Simplified'] = ccle_mutations['Variant_Classification_Simplified'].str.replace("Frame_Shift_Del","Deletion")
ccle_mutations['Variant_Classification_Simplified'] = ccle_mutations['Variant_Classification_Simplified'].str.replace("Nonsense_Mutation","Nonsense")
ccle_mutations['Variant_Classification_Simplified'] = ccle_mutations['Variant_Classification_Simplified'].str.replace("Frame_Shift_Ins","Insertion")
ccle_mutations['Variant_Classification_Simplified'] = ccle_mutations['Variant_Classification_Simplified'].str.replace("In_Frame_Del","Deletion")
ccle_mutations['Variant_Classification_Simplified'] = ccle_mutations['Variant_Classification_Simplified'].str.replace("De_novo_Start_OutOfFrame","Other")
ccle_mutations['Variant_Classification_Simplified'] = ccle_mutations['Variant_Classification_Simplified'].str.replace("In_Frame_Ins","Insertion")
ccle_mutations['Variant_Classification_Simplified'] = ccle_mutations['Variant_Classification_Simplified'].str.replace("Start_Codon_SNP","Other")
ccle_mutations['Variant_Classification_Simplified'] = ccle_mutations['Variant_Classification_Simplified'].str.replace("Nonstop_Mutation","Nonstop")
ccle_mutations['Variant_Classification_Simplified'] = ccle_mutations['Variant_Classification_Simplified'].str.replace("Intron","Other")
ccle_mutations['Variant_Classification_Simplified'] = ccle_mutations['Variant_Classification_Simplified'].str.replace("Stop_Codon_Del","Deletion")
ccle_mutations['Variant_Classification_Simplified'] = ccle_mutations['Variant_Classification_Simplified'].str.replace("Stop_Codon_Ins","Insertion")
ccle_mutations['Variant_Classification_Simplified'] = ccle_mutations['Variant_Classification_Simplified'].str.replace("Start_Codon_Del","Deletion")
ccle_mutations['Variant_Classification_Simplified'] = ccle_mutations['Variant_Classification_Simplified'].str.replace("IGR","Other")
ccle_mutations['Variant_Classification_Simplified'] = ccle_mutations['Variant_Classification_Simplified'].str.replace("Start_Codon_Ins","Insertion")
ccle_mutations['Variant_Classification_Simplified'] = ccle_mutations['Variant_Classification_Simplified'].str.replace("5'Flank","Other")
ccle_mutations['Variant_Classification_Simplified'] = ccle_mutations['Variant_Classification_Simplified'].str.replace("3'UTR","Other")
ccle_mutations['Variant_Classification_Simplified'] = ccle_mutations['Variant_Classification_Simplified'].str.replace("5'UTR","Other")



In [ ]:
# map DepMap_ID to model_id
ccle_mutations = ccle_mutations.merge(cell_line_mapping[['BROAD_ID','model_id']],
                     left_on = 'DepMap_ID',
                     right_on = 'BROAD_ID',
                     how = 'left')

# map Entrez_Gene_Id to gene_id
ccle_mutations = ccle_mutations.merge(gene_mapping[['entrez_id','gene_id']],
                     left_on = 'Entrez_Gene_Id',
                     right_on = 'entrez_id',
                     how = 'left')

# remove duplicates
print(ccle_mutations.shape)
ccle_mutations.drop_duplicates(subset=['gene_id','model_id','Variant_Classification_Simplified'], inplace=True)
print(ccle_mutations.shape)


print('percentage of na in model_id: ', ccle_mutations['model_id'].isna().sum()/ccle_mutations.shape[0])
print('percentage of na in gene_id: ', ccle_mutations['gene_id'].isna().sum()/ccle_mutations.shape[0])


In [ ]:
matrix_mutations = ccle_mutations.pivot_table(index = 'model_id',
                          columns = 'gene_id',
                        values = 'Variant_Classification_Simplified',
                        aggfunc='first')

matrix_mutations_binary = matrix_mutations.copy()
print(matrix_mutations.shape)

matrix_mutations[matrix_mutations.isna()] = 0  
matrix_mutations_binary[matrix_mutations_binary.notna()] = 1  
matrix_mutations_binary[matrix_mutations_binary.isna()] = 0  




matrix_mutations_frequency = ccle_mutations.pivot_table(index = 'model_id',
                          columns = 'gene_id',
                        values = 'Variant_Classification_Simplified',
                        aggfunc='count')
matrix_mutations_frequency[matrix_mutations_frequency.isna()] = 0  
print(matrix_mutations.shape)

matrix_mutations.head(10)



matrix_mutations.to_csv('./matrix_mutations.csv', index = True)
matrix_mutations_binary.to_csv('./matrix_mutations_binary.csv', index = True)
matrix_mutations_binary.to_csv('./matrix_mutations_binary_pca.csv', header=False, index = False)

In [ ]:
gdsc_drugResponse = pd.read_excel(r'../input/gdsc2-fitted-dose-response-25feb20/GDSC2_fitted_dose_response_25Feb20.xlsx')
print(gdsc_drugResponse.shape)
gdsc_drugResponse.head()

In [ ]:
gdsc_drugResponse.rename(columns = {'SANGER_MODEL_ID':'model_id', 'DRUG_ID':'drug_id'}, inplace=True)

matrix_drug_response = gdsc_drugResponse.pivot_table(columns='drug_id',
                             index='model_id',
                             values='LN_IC50')
matrix_drug_response[matrix_drug_response.isna()] = 0  

matrix_drug_response.head()
matrix_drug_response.shape

In [ ]:
matrix_drug_response.to_csv('./matrix_drug_response.csv', index=True)


In [ ]:
ccle_miRNA_original = pd.read_csv(r'../input/cclemirna/CCLE_miRNA_20181103.csv')
ccle_miRNA = ccle_miRNA_original.copy()
# print(ccle_miRNA.shape)
ccle_miRNA.head()


In [ ]:
miRNA_t = ccle_miRNA.T
print(miRNA_t.shape)

In [ ]:
miRNA_t.head()
miRNA_t.to_csv('./miRNA_t.csv', header=False, index = False)

In [ ]:
miRNA_trans = pd.read_csv(r'../input/mirna-trans/miRNA_trans.csv')
miRNA_trans.head()

In [ ]:
# map CCLE_ID to model_id
miRNA_trans.rename(columns={'Unnamed: 0':'CCLE_ID'},inplace=True)
matrix_mirRNA = miRNA_trans.merge(cell_line_mapping[['CCLE_ID','model_id']],
                     left_on = 'CCLE_ID',
                     right_on = 'CCLE_ID',
                     how = 'left')
#Removing NA values (meaning no model ID matching)

print('percentage of na in model_id: ', matrix_mirRNA['model_id'].isna().sum()/matrix_mirRNA.shape[0])
print(matrix_mirRNA.shape)

matrix_mirRNA.dropna(subset=['model_id'], inplace=True)
matrix_mirRNA = matrix_mirRNA.iloc[:,1:]
matrix_mirRNA = matrix_mirRNA[[list(matrix_mirRNA.columns)[-1]]+ list(matrix_mirRNA.columns[:-1])]
matrix_mirRNA.to_csv('./matrix_mirRNA.csv', index = False)
matrix_mirRNA.to_csv('./matrix_mirRNA_pca.csv', header=False, index = False)

# In[23]:


print(matrix_mirRNA.shape)
matrix_mirRNA.head()
